In [3]:
import random
import csv
import sys
csv.field_size_limit(sys.maxsize)
import pandas as pd

In [4]:
import json

In [5]:

def convert_wmt ():
    #TODO there's definetely a way to clean this up
    with open("translation_data/news-commentary-v15.en-fr.tsv") as file:
        tsv_file = csv.reader(file, delimiter="\t")
        i = 0
        res = []
        # generate data for everything, shuffle and sample later
        for f in tsv_file:
            if i == 100000:
                break
            res.append(f)
            i = i+1
    random.shuffle(res)
    print("GOT HERE SOMEHOW")
    res = res[:10000]
    tmpdf = pd.DataFrame(res)
    tmpdf['en'] = tmpdf[0]
    tmpdf['fr'] = tmpdf[1]
    del tmpdf[0]
    del tmpdf[1]
    with open('mt-data/use/fr-en.fr', 'w') as f:
        for line in tmpdf['fr']:
            f.write(line)
            f.write('\n')
    with open('mt-data/use/fr-en.en', 'w') as f:
        for line in tmpdf['en']:
            f.write(line)
            f.write('\n')
    
    return tmpdf

In [6]:
def load_cands(fname):
    data = []
    with open(fname, 'r') as file:
        while True:
            line = file.readline()
            #print(line)
            # if line is empty
            # end of file is reached
            if not line or len(line)<3:
                break

            data.append(json.loads(line))
    return data

In [7]:
german= load_cands("./candoutputs/explodedmt1n_en-de_bfs_recom_4_80_False_0.4_True_False_4_5_rcb_0.904_0.0_0.9.jsonl")

In [5]:
french = load_cands("./candoutputs/explodedmtn1_fr-en_bfs_recom_4_-1_False_0.4_True_False_4_5_rcb_0.904_0.0_0.9.jsonl")

In [9]:
def lset_tocominp(lset):
    res = []
    for g in lset:
        if len(g['cands'])<32:
            continue
        samp = random.sample(g['cands'], 32)+[g['ref']]
        for s in samp:
            tmp = {}
            tmp['src'] = g['inp']
            tmp['hyp'] = s
            tmp['ref'] = g['ref']
            res.append(tmp)
    return res

In [10]:
cominpde = lset_tocominp(german)

In [12]:
cpfr = pd.DataFrame(cominpde)

In [14]:
cpfr.to_csv("./processeddata/cpgdata.csv")

In [18]:
cpfr[:64]['hyp']

0     After all, as an investigative reporter in the...
1     After all, as a field investigative journalist...
2     After all, as an investigative reporter in the...
3     After all, as a journalist investigating the c...
4     After all, as a journalist investigating in th...
                            ...                        
59    After all, as a journalist investigating the c...
60    After all, as an investigative reporter in the...
61    After all, as an investigative journalist in t...
62    After all, as an investigative reporter in the...
63    After all, as an investigative journalist on t...
Name: hyp, Length: 64, dtype: object

In [3]:
cpfr = pd.read_csv("./processeddata/cpfdata1.csv")

In [4]:
cpfr = cpfr.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [5]:
cpfr

,src,hyp,ref,score
0,"Après tout, en tant que journaliste d'investig...","After all, as an investigative reporter in the...","After all, as a campaigning investigative jour...",0.423756
1,"Après tout, en tant que journaliste d'investig...","After all, as a field investigative journalist...","After all, as a campaigning investigative jour...",0.500724
2,"Après tout, en tant que journaliste d'investig...","After all, as an investigative reporter in the...","After all, as a campaigning investigative jour...",0.377539
3,"Après tout, en tant que journaliste d'investig...","After all, as a journalist investigating the c...","After all, as a campaigning investigative jour...",0.240281
4,"Après tout, en tant que journaliste d'investig...","After all, as a journalist investigating in th...","After all, as a campaigning investigative jour...",0.253478
...,...,...,...,...
309135,De grosses erreurs ont été commises qui ont en...,Large errors were committed which further fuel...,"Big mistakes were made, fueling further violen...",0.798258
309136,De grosses erreurs ont été commises qui ont en...,Major mistakes were made and the violence was ...,"Big mistakes were made, fueling further violen...",0.803969
309137,De grosses erreurs ont été commises qui ont en...,Big mistakes have been made and the violence c...,"Big mistakes were made, fueling further violen...",0.747567
309138,De grosses erreurs ont été commises qui ont en...,Big mistakes have been made and they have furt...,"Big mistakes were made, fueling further violen...",0.864925


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

2022-10-17 17:02:59.438560: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-17 17:02:59.438585: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"


In [25]:
def get_mbart_nll(inp, out, inptok, labtok, mod, dev):

    inputs = inptok(inp, return_tensors="pt").to(dev)
    with labtok.as_target_tokenizer():
        labels = labtok(out, return_tensors="pt").to(dev)

    # forward pass
    output = mod(**inputs, labels=labels.input_ids)
    #print(type(labels))
    #print(labels.attention_mask)
    return output.loss

def rescore_cands(dset, hyplist, srclist):
    device = "cuda:1" if torch.cuda.is_available() else "cpu"
    if "de" in dset:
        mname = "facebook/mbart-large-50-one-to-many-mmt"
        src_l = "en_XX"
        tgt_l = "de_DE"
    else:
        mname = "facebook/mbart-large-50-many-to-one-mmt"
        src_l = "fr_XX"
        tgt_l = "en_XX"
    inptok = AutoTokenizer.from_pretrained(mname)
    labtok = AutoTokenizer.from_pretrained(mname, src_lang=src_l, tgt_lang=tgt_l)
    mod = AutoModelForSeq2SeqLM.from_pretrained(mname)
    mod.to(device)
    print("rescoring candidates")
    i = 0
    result = []
    for i in range(0, len(hyplist)):
        if i%500==0:
            print(i)
        try:
            result.append(float(get_mbart_nll(srclist[i], hyplist[i], inptok, labtok, mod, device)))
        except:
            result.append(0)
            continue
        #print(i)
        i+=1
    del inptok
    del labtok
    del mod
    torch.cuda.empty_cache()
    return result

In [16]:
# cache dir for cometqe model
cometqe_dir = "./cometqemodel"
# can alternatively use wmt21-comet-qe-mqm
cometqe_model = "wmt20-comet-qe-da"

def get_cometqe_scores(hyps, srcs, commodel):
    cometqe_input = [{"src": src, "mt": mt} for src, mt in zip(srcs, hyps)]
    # sentence-level and corpus-level COMET
    outputs = commodel.predict(
        cometqe_input, batch_size=32, progress_bar=True
    )
    torch.cuda.empty_cache()
    return outputs

In [12]:
from comet import download_model, load_from_checkpoint

cometqe_path = download_model(cometqe_model, cometqe_dir)
model = load_from_checkpoint(cometqe_path).to(device)

wmt20-comet-qe-da is already in cache.
Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.


In [17]:
qescores = get_cometqe_scores(cpfr['hyp'], cpfr['src'], model)

In [19]:
len(qescores[0])

309140

In [20]:
cpfr['comscores'] = qescores[0]

In [23]:
del model 
del cometqe_path

In [26]:
res = rescore_cands('fr-en', cpfr['hyp'], cpfr['src'])
cpfr['posthoc'] = res

rescoring candidates
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
66000
66500
67000
67500
68000
68500
69000
69500
70000
70500
71000
71500
72000
72500
73000
73500
74000
74500
75000
75500
76000
76500
77000
77500
78000
78500
79000
79500
80000
80500
81000
81500
82000
82500
83000
8

In [27]:
cpfr.to_csv("./processeddata/cpfdata2.csv")

In [28]:
cpfr

,src,hyp,ref,score,comscores,posthoc
0,"Après tout, en tant que journaliste d'investig...","After all, as an investigative reporter in the...","After all, as a campaigning investigative jour...",0.423756,0.001024,0.534086
1,"Après tout, en tant que journaliste d'investig...","After all, as a field investigative journalist...","After all, as a campaigning investigative jour...",0.500724,0.187095,0.493034
2,"Après tout, en tant que journaliste d'investig...","After all, as an investigative reporter in the...","After all, as a campaigning investigative jour...",0.377539,0.009930,0.544490
3,"Après tout, en tant que journaliste d'investig...","After all, as a journalist investigating the c...","After all, as a campaigning investigative jour...",0.240281,0.006175,0.509931
4,"Après tout, en tant que journaliste d'investig...","After all, as a journalist investigating in th...","After all, as a campaigning investigative jour...",0.253478,0.000042,0.760619
...,...,...,...,...,...,...
309135,De grosses erreurs ont été commises qui ont en...,Large errors were committed which further fuel...,"Big mistakes were made, fueling further violen...",0.798258,0.331076,0.684784
309136,De grosses erreurs ont été commises qui ont en...,Major mistakes were made and the violence was ...,"Big mistakes were made, fueling further violen...",0.803969,0.414651,0.797772
309137,De grosses erreurs ont été commises qui ont en...,Big mistakes have been made and the violence c...,"Big mistakes were made, fueling further violen...",0.747567,0.365663,0.942066
309138,De grosses erreurs ont été commises qui ont en...,Big mistakes have been made and they have furt...,"Big mistakes were made, fueling further violen...",0.864925,0.487513,0.732962
